In [62]:
# import PyPDF2
import fitz
# import copy
import os
import re

In [3]:
file = "en_2022-2024 WAG COP.pdf"

In [5]:
def writeImage(image_path, image_bytes, count=1):
    # string = "page80_element_2.405_1.png"
    if count == 1:
        image_path = image_path.replace(".png", f"_{count}.png")
    else:
        image_path = re.sub(r"(\d?).png", count+".png", image_path)
    if os.path.exists(image_path):
        writeImage(image_path, image_bytes, count+1)
    else:
        with open(image_path, "wb") as image_file:
            image_file.write(image_bytes)

def getPageLayout(page):
    bbox = page.bleedbox
    if bbox.x1-bbox.y1 < 0:
        return "portrait"
    else:
        return "landscape"
    
def extract_text_and_images(pdf_path, output_folder, target_page, regex_pattern):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Open the PDF file
    pdf_document = fitz.open(pdf_path)
    
    # Check if the target page is within the range of pages
    if target_page < 0 or target_page >= len(pdf_document):
        print("Error: Invalid target page number.")
        return
    
    page = pdf_document.load_page(target_page)
    page_layout = getPageLayout(page)

    # Extract text from the page
    page_text = page.get_text()
    # print(page_text)
    found_text = re.findall(regex_pattern, page_text)
    print("FOUND", found_text)
    # Extract images and their positions
    images = page.get_images(full=True)
    
    # Loop through images on the page
    for img_info in images:
        xref = img_info[0]
        base_image = pdf_document.extract_image(xref)
        image_bytes = base_image["image"]

        # Get the text closest and above the image
        bbox_image = page.get_image_bbox(img_info[7])
        # print("bbox img", bbox_image)

        if (bbox_image.bottom_right - bbox_image.top_left)[0] < 75:
            # assumed to be a 
            print("assuming", img_info[7], "is a judge symbol")
            continue


        found = ""
    
        optionA = []
        closestx = 1e6
        for item in found_text:
            for field in page.search_for(item):
                if page_layout == "portrait":
                    text_y1 = field.x1
                    text_x1 = abs(field.y0 - page.bleedbox.y1)
                else:
                    text_y1 = field.y1
                    text_x1 = field.x1
                x = round(bbox_image.y0 - round(text_y1))
                # print(field)
                # print(bbox_image)
                if x < 0:
                    continue
                if x < 15:
                    # In this case the number is too close to the picture and it is assumed its a number from the points; e.g. "3.20 P." instead of element 3.20
                    continue
                if x == closestx:
                    optionA.append(item)
                    closestx = x
                elif x < closestx:
                    optionA = [item]
                    closestx = x
                # print(optionA)
                for option in optionA:
                    # print(field)
                    # print(bbox_image.x0, text_x1, bbox_image.x1)
                    if bbox_image.x0 < text_x1 < bbox_image.x1:
                        # print("hurray", item)
                        found = option
                        pass
            
        if found != "":
            image_path = os.path.join(output_folder, f"page{target_page}_element_{found}.png")
            
            writeImage(image_path, image_bytes)
        else:
            print("warning! could not process ", img_info[7])
    pdf_document.close()

# Example usage
pdf_file_path = file
output_folder = "../data/images/"
target_page = 104  # Specify the page number you want to extract text and images from
regex_pattern = re.compile(r"(\d+\.\d+)\s?\n")  # Specify your regex pattern here
# extract_text_and_images(pdf_file_path, output_folder, target_page, regex_pattern)


In [277]:
pdf_document = fitz.open(file)
page = pdf_document.load_page(page_id=80)
getPageLayout(page=page)

'portrait'

In [278]:
page_text = page.get_text()
# print(page_text)
found_text = re.findall(r"(\d+\.\d+)\s+\n?(?!P\.)", page_text)
print(found_text)

['2.000', '2.105', '2.205', '2.305', '2.405', '2.505', '2.605']


In [279]:
# left, up, right, bottom
# clip = fitz.Rect(28.19099998474121 - 50, 113.76000213623047 - 50, 145.79100036621094 + 50, 186.24000549316406)
clip = page.get_image_bbox("Im506")
# clip = fitz.Rect(168.59100341796875, 103.19999694824219 - 50, 277.0710144042969, 162.72000122070312)
pix = page.get_pixmap(matrix=fitz.Matrix(2, 2), clip=clip)
pix.save("test.png")

In [134]:
def findText(page, image, found_text, page_layout):
    found = ""
    optionA = []
    closestx = 1e6
    bbox_image = page.get_image_bbox(image[7])
    print("bbox", bbox_image)
    for item in found_text:
        for field in page.search_for(item):
            print(item, field)
            # some pages have a portrait mode, other landscape mode. 
            # This is important when matching the text with images. 
            # if the page is in portrait mode, we need to do a new calculation to make sure the matching works.
            if page_layout == "portrait":
                text_y1 = field.x1
                text_x1 = abs(field.y0 - page.bleedbox.y1)
                # text_x1 = field.y0
            elif page_layout == "landscape":
                text_y1 = field.y1
                text_x1 = field.x1
            elif page_layout == "other":
                text_y1 = field.y1
                text_x1 = abs( field.x1 - page.bleedbox.y1)
            diff = round(bbox_image.y0 - round(text_y1 /10)* 10)
            if diff < 0:
                continue
            if diff < 10:
                # In this case the number is too close to the picture and it is assumed its a number from the points; e.g. "3.20 P." instead of element 3.20
                print("yes")
                continue
            if diff == closestx:
                optionA.append([item, text_x1])
                closestx = diff
            elif diff < closestx:
                optionA = [[item, text_x1]]
                closestx = diff
    
    print(optionA)
    for option, text_x1 in optionA:
        print(option, bbox_image.x0, text_x1, bbox_image.x1)
        if bbox_image.x0 - 35 < text_x1 < bbox_image.x1 :
            found = option
            print("found", found)
   
    
    if found == "":
        distance = 1e6
        for option, text_x1 in optionA:
            print(option, bbox_image.x1 - text_x1 , distance)
            if 0 < bbox_image.x1 - text_x1 < distance:
                distance = bbox_image.x1 - text_x1
                found = option

    return found

findText(page, page.get_images()[1], found_text, "portrait" )

bbox Rect(686.510009765625, 295.67999267578125, 794.27001953125, 353.5199890136719)
4.000 Rect(35.74916076660156, 495.4164733886719, 46.94337844848633, 520.469482421875)
4.103 Rect(59.855918884277344, 793.6513671875, 68.77107238769531, 813.6295166015625)
4.203 Rect(59.855918884277344, 671.2512817382812, 68.77107238769531, 691.2293701171875)
4.303 Rect(59.855918884277344, 536.611572265625, 68.77107238769531, 556.589599609375)
4.403 Rect(59.855918884277344, 476.01507568359375, 68.77107238769531, 495.9931640625)
4.503 Rect(59.855918884277344, 422.0206298828125, 68.77107238769531, 441.99871826171875)
4.603 Rect(59.855918884277344, 273.52227783203125, 68.77107238769531, 293.5003662109375)
4.104 Rect(160.29592895507812, 793.6513671875, 169.21107482910156, 813.6295166015625)
4.304 Rect(160.29592895507812, 536.6099243164062, 169.21107482910156, 556.5880126953125)
4.404 Rect(160.29592895507812, 476.0093994140625, 169.21107482910156, 495.98748779296875)
4.504 Rect(160.29592895507812, 422.0093994

'4.605'

In [147]:
from functions import *

In [210]:
metadata = loadJson("../data/images/uneven bars/metadata.json")

In [212]:
df = pd.read_json("../data/images/uneven bars/metadata.json")
df

,img_id,page,bbox,element,filename
0,Im369,73,"{'x0': 37.84999465942383, 'y0': 424.3404541015...",1.102,data/images/uneven bars/page73_element_1.102_I...
1,Im368,73,"{'x0': 33.999996185302734, 'y0': 114.689727783...",1.101,data/images/uneven bars/page73_element_1.101_I...
2,Im370,73,"{'x0': 33.999996185302734, 'y0': 288.539276123...",1.102,data/images/uneven bars/page73_element_1.102_I...
3,Im372,73,"{'x0': 159.44998168945312, 'y0': 290.840057373...",1.202,data/images/uneven bars/page73_element_1.202_I...
4,Im385,74,"{'x0': 168.44998168945312, 'y0': 318.500366210...",1.204,data/images/uneven bars/page74_element_1.204_I...
...,...,...,...,...,...
245,Im885,105,"{'x0': 292.9999694824219, 'y0': 116.4501342773...",6.309,data/images/uneven bars/page105_element_6.309_...
246,Im891,105,"{'x0': 438.4499816894531, 'y0': 219.1392822265...",6.409,data/images/uneven bars/page105_element_6.409_...
247,Im876,105,"{'x0': 699.8099365234375, 'y0': 130.3596191406...",6.609,data/images/uneven bars/page105_element_6.609_...
248,Im887,105,"{'x0': 292.9999694824219, 'y0': 224.451171875,...",6.309,data/images/uneven bars/page105_element_6.309_...


In [280]:
for x in df.groupby("element").agg(list).iterrows():
    print(x[1])
    newBbox = {"x0": 0, "y0":0, "x1":0, "y1":0}
    for bbox in x[1]["bbox"]:
        print(bbox)
        for axis, value in bbox.items():
            # init
            if newBbox[axis] == 0:
                newBbox[axis] = value
            else:
                # get the outer boundaries combined
                if axis == "x0" or axis == "y0":
                    if newBbox[axis] < value:
                        newBbox[axis] = value
                else:
                    if newBbox[axis] > value:
                        newBbox[axis] = value
    assert len(set(x[1]["page"])) == 1
    print(x[0])
    print(newBbox)
    break

img_id                                                [Im368]
page                                                     [73]
bbox        [{'x0': 33.999996185302734, 'y0': 114.68972778...
filename    [data/images/uneven bars/page73_element_1.101_...
Name: 1.101, dtype: object
{'x0': 33.999996185302734, 'y0': 114.68972778320312, 'x1': 137.3197021484375, 'y1': 166.92755126953125}
1.101
{'x0': 33.999996185302734, 'y0': 114.68972778320312, 'x1': 137.3197021484375, 'y1': 166.92755126953125}


In [281]:
subdf = df.groupby("element").agg(list)
subdf

,img_id,page,bbox,filename
element,,,,
1.101,[Im368],[73],"[{'x0': 33.999996185302734, 'y0': 114.68972778...",[data/images/uneven bars/page73_element_1.101_...
1.102,"[Im369, Im370]","[73, 73]","[{'x0': 37.84999465942383, 'y0': 424.340454101...",[data/images/uneven bars/page73_element_1.102_...
1.103,"[Im388, Im389]","[74, 74]","[{'x0': 46.150001525878906, 'y0': 126.49002075...",[data/images/uneven bars/page74_element_1.103_...
1.104,[Im379],[74],"[{'x0': 33.999996185302734, 'y0': 313.29904174...",[data/images/uneven bars/page74_element_1.104_...
1.106,[Im395],[75],"[{'x0': 52.002994537353516, 'y0': 292.33978271...",[data/images/uneven bars/page75_element_1.106_...
...,...,...,...,...
6.605,[Im831],[102],"[{'x0': 699.8099365234375, 'y0': 97.0094299316...",[data/images/uneven bars/page102_element_6.605...
6.608,[Im871],[104],"[{'x0': 699.6099243164062, 'y0': 124.859924316...",[data/images/uneven bars/page104_element_6.608...
6.609,[Im876],[105],"[{'x0': 699.8099365234375, 'y0': 130.359619140...",[data/images/uneven bars/page105_element_6.609...


In [276]:
 for x in subdf[subdf["page"].str.len() ==5]["bbox"][2.105]:
    print(x)

{'x0': 28.371000289916992, 'y0': 339.6000061035156, 'x1': 131.1510009765625, 'y1': 350.6400146484375}
{'x0': 28.371000289916992, 'y0': 383.760009765625, 'x1': 131.1510009765625, 'y1': 394.32000732421875}
{'x0': 28.371000289916992, 'y0': 372.7200012207031, 'x1': 131.1510009765625, 'y1': 383.760009765625}
{'x0': 28.371000289916992, 'y0': 361.67999267578125, 'x1': 131.1510009765625, 'y1': 372.7200012207031}
{'x0': 28.371000289916992, 'y0': 350.6399841308594, 'x1': 131.1510009765625, 'y1': 361.67999267578125}


In [282]:
[bbox["x0"] for bbox in subdf[subdf["page"].str.len() ==5]["bbox"][2.105]]

[28.371000289916992,
 28.371000289916992,
 28.371000289916992,
 28.371000289916992,
 28.371000289916992]

In [285]:
[m for m in metadata if m["page"]==80]

[{'img_id': 'Im494',
  'page': 80,
  'bbox': {'x0': 582.77099609375,
   'y0': 124.56000518798828,
   'x1': 690.8309936523438,
   'y1': 195.60000610351562},
  'element': '2.503',
  'filename': 'data/images/uneven bars/page80_element_2.503_Im494.png'},
 {'img_id': 'Im496',
  'page': 80,
  'bbox': {'x0': 456.1109924316406,
   'y0': 144.66000366210938,
   'x1': 563.990966796875,
   'y1': 193.32000732421875},
  'element': '2.403',
  'filename': 'data/images/uneven bars/page80_element_2.403_Im496.png'},
 {'img_id': 'Im500',
  'page': 80,
  'bbox': {'x0': 37.55099868774414,
   'y0': 351.3599853515625,
   'x1': 140.51100158691406,
   'y1': 408.239990234375},
  'element': '2.104',
  'filename': 'data/images/uneven bars/page80_element_2.104_Im500.png'},
 {'img_id': 'Im497',
  'page': 80,
  'bbox': {'x0': 328.4909973144531,
   'y0': 133.1999969482422,
   'x1': 404.57098388671875,
   'y1': 193.86000061035156},
  'element': '2.303',
  'filename': 'data/images/uneven bars/page80_element_2.303_Im497.

In [283]:
# left, up, right, bottom
# clip = fitz.Rect(28.19099998474121 - 50, 113.76000213623047 - 50, 145.79100036621094 + 50, 186.24000549316406)
# clip = page.get_image_bbox("Im431")
clip = fitz.Rect(28, 339.6000061035156-5, 131.1510009765625, 394.6400146484375)
pix = page.get_pixmap(matrix=fitz.Matrix(2, 2), clip=clip)
pix.save("test.png")

In [247]:
clip

Rect(28.371000289916992, 339.6000061035156, 131.1510009765625, 350.6400146484375)